In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

import pickle
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import files
uploaded = files.upload()

Saving input (1).txt to input (1).txt


**Load and Preprocess the data**

In [3]:
file = open("input.txt" , "r" , encoding = "utf8")

# storing file in list
lines = []
for i in file:
    lines.append(i)

# Converting list to string
data  = ""
for i in lines:
    data = ' '.join(lines)

# replacing unnecessary stuff with space
data = data.replace('\n' ,  ' ').replace('\r', ' ').replace('\ufeff' , ' ').replace('“', ' ').replace('”', ' ' )

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

check the length of the data used for pre-processinng


In [ ]:
len(data)

**Apply Tokenization:**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl' , 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

In [ ]:
len(sequence_data)

In [ ]:
vocab_size = len(tokenizer.word_index) +  1
print(vocab_size)

In [ ]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The length of the sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

In [ ]:
X =[]
Y =[]
for i in sequences:
    X.append(i[0:3])
    Y.append(i[3])

X = np.array(X)
Y = np.array(Y)

In [ ]:
print("Data:   ", X[:10])
print("Response:   ", Y[:10])

In [ ]:
Y = to_categorical(Y, num_classes = vocab_size)
Y[:5]

**Creating the model**

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))

#feeding it into the LSTM layer
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))

#Defining  our neural network pattern
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

# **Plotting our model**
*to know the detailed visual representation*

In [ ]:
from tensorflow import keras
from keras.utils import plot_model
# Plotting the model for understanding
keras.utils.plot_model(model, to_file = 'plot.png', show_layer_names = True)

# Building our model

In [ ]:
#Training the Model

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy" , optimizer=Adam(learning_rate=0.001))
model.fit(X, Y, epochs=10, batch_size=64, callbacks=[checkpoint])

# Prediction

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#loading the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np. argmax(model.predict(sequence))
    predicted_word = ""

    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break

    print(predicted_word)
    return predicted_word

# *Output:*

In [ ]:
while(1):
    text = input("Enter your line from the text file: ")
    if text == "0":
        print("Execution completed!")
        break
    else:
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)

            Predict_Next_Words(model, tokenizer, text)
        except Exception as e:
            print("Error occured:  ", e)
            continue